# Naver Map Scraper v1.0.0
- Last Updated @2023-09-30
- Contributor : Tony Park (https://heytech.tistory.com/)

# Package Import

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
from tqdm import tqdm

In [3]:
# 최신 크롬 드라이버 사용하도록 세팅: 현재 OS에 설치된 크롬 브라우저 버전에 맞게 cache에 드라이버 설치
from selenium.webdriver.chrome.service import Service
service = Service(ChromeDriverManager().install())

# DB scan

In [4]:
FILENAME = './콘텐츠 수집 DB_230930.xlsx'
SHEET_NAME = 'DB_TTL_230930'

DB = pd.read_excel(FILENAME, sheet_name=SHEET_NAME)

In [5]:
DB_URL = DB['네이버지도URL']
DB_URL.dropna(inplace=True)
DB_URL = list(DB_URL)

In [6]:
COLUMN_NAMES = [
    'name',
    'category',
    'address',
    'days',
    'time1',
    'time2',
    'tel',
    'desc1',
    'desc2',
    'desc3',
]

# Scrap v1.0.1

In [11]:
name_list = []
category_list = []
address_list = []
time_mon_list = []
time_tue_list = []
time_wed_list = []
time_thr_list = []
time_fri_list = []
time_sat_list = []
time_sun_list = []
time_txt_list = []
tel_list = []
desc1_list = []
desc2_list = []
desc3_list = []

In [12]:
day_list = ['월','화','수','목','금','토','일']

In [13]:
for i in tqdm(range(len(DB_URL))):
    try:
        db_scrap_new = pd.DataFrame()
        driver = webdriver.Chrome(service=service)
        
        URL = DB_URL[i]
        
        # 크롬 드라이버를 통해 지정한 URL의 웹 페이지 오픈
        driver.get(URL)
        driver.implicitly_wait(3) # seconds

        # iframe 이동
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]'))
        
        # 영업시간 모두 보기
        try:
            driver.implicitly_wait(5) # seconds
            TIME_DETAIL_BTN = '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/div/div[3]/div/a/div[1]'
            driver.find_element(By.XPATH, TIME_DETAIL_BTN).click()
        except: 
            pass
        
        # 영업시간 모두 보기
        try:
            driver.implicitly_wait(5) # seconds
            TIME_DETAIL_BTN = '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/div/div[2]/div/a/div'
            driver.find_element(By.XPATH, TIME_DETAIL_BTN).click()
        except: 
            pass
        
        # 상세정보 보기
        try:
            driver.implicitly_wait(5) # seconds
            DESC_DETAIL_BTN = '//*[@id="app-root"]/div/div/div/div[7]/div/div[2]/div/div/div[9]/div/a/span[1]'
            driver.find_element(By.XPATH, DESC_DETAIL_BTN).click()
        except: 
            pass

        # 페이지 소스 추출
        html_source = driver.page_source
        soup_source = BeautifulSoup(html_source, 'html.parser')
        

        #-------- 콘텐츠명 --------#
        try:
            TITLE_CLASS = 'Fc1rA'
            name_source = soup_source.find_all(class_ = TITLE_CLASS)
            name = name_source[0].get_text()
        except:
            name = ''
        print(f"name: {name}")
        name_list.append(name)
    
        #-------- 카테고리 --------#
        try:
            CATEGORY_CLASS = 'DJJvD'
            category_source = soup_source.find_all(class_ = CATEGORY_CLASS)
            category = category_source[0].get_text()
        except:
            category = ''
        # print(f"category: {category}")
        category_list.append(category)
        
        
        #-------- 주소 --------#
        try:
            ADDRESS_CLASS = 'LDgIH'
            address_source = soup_source.find_all(class_ = ADDRESS_CLASS)
            address = address_source[0].get_text()
        except:
            address = ''
        # print(f"address: {address}")
        address_list.append(address)
        
        
        #-------- 영업시간 ---------#
        try:
            TIME_CLASS = 'H3ua4'
            time_source = soup_source.find_all(class_ = TIME_CLASS)
            
            DAY_CLASS = 'i8cJw'
            day_source = soup_source.find_all(class_ = DAY_CLASS)
            
            time_dict = {}
            
            
            if day_source[0].get_text() == '매일':
                for day in day_list:
                    time_dict[day] = time_source[0].get_text()
            else:
                for day, time in zip(day_source, time_source):
                    day = day.get_text()[0]
                    time = time.get_text()
                    time_dict[day] = time
                    
            print(f"time_dict : {time_dict}")
            time_mon_list.append(time_dict['월'])
            time_tue_list.append(time_dict['화'])
            time_wed_list.append(time_dict['수'])
            time_thr_list.append(time_dict['목'])
            time_fri_list.append(time_dict['금'])
            time_sat_list.append(time_dict['토'])
            time_sun_list.append(time_dict['일'])
        
        except:
            time_mon_list.append('')
            time_tue_list.append('')
            time_wed_list.append('')
            time_thr_list.append('')
            time_fri_list.append('')
            time_sat_list.append('')
            time_sun_list.append('')
            
        try:
            TIME1_CLASS = 'U7pYf'
            time1_source = soup_source.find_all(class_ = TIME1_CLASS)
            time1 = time1_source[0].get_text()
        except:
            time1 = ''
        # print(f"time1: {time1}")
        time_txt_list.append(time1)
            
        #-------- 전화번호 ---------#
        try:
            TEL_CLASS = 'xlx7Q'
            tel_source = soup_source.find_all(class_ = TEL_CLASS)
            tel = tel_source[0].get_text()
        except:
            tel = ''
        # print(f"tel: {tel}")
        tel_list.append(tel)
        
        
        #-------- 간략소개 ---------#
        try:
            desc_short_source = soup_source.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['O8qbU'])
            desc_short = desc_short_source[0].get_text()
        except Exception as e:
            print(f"Error : {e}")
            desc_short = ''
        # print(f"desc_short : {desc_short}")
        desc1_list.append(desc_short)
        
        
        #-------- 상세소개 ---------#
        try:
            DESC_DETAIL_CLASS = 'zPfVt'
            desc_detail_source = soup_source.find_all(class_ = DESC_DETAIL_CLASS)
            desc_detail = desc_detail_source[-1].get_text()
        except:
            desc_detail = ''
        # print(f"desc_detail : {desc_detail}")
        desc2_list.append(desc_detail)

        #-------- 상세소개2 ---------#
        try:
            DESC_DETAIL2_CLASS = 'y6tNq'
            desc_detail2_source = soup_source.find_all(class_ = DESC_DETAIL2_CLASS)
            desc_detail2 = desc_detail2_source[0].get_text()
        except:
            desc_detail2 = ''
        # print(f"desc_detail : {desc_detail2}")
        desc3_list.append(desc_detail2)
        driver.quit()
        
    except Exception as e:
        print(f"error : {e}")

  1%|▏                                        | 1/182 [00:20<1:02:46, 20.81s/it]

name: 을왕리해수욕장


  1%|▍                                        | 2/182 [00:41<1:01:46, 20.59s/it]

name: 왕산해수욕장


  2%|▋                                        | 3/182 [01:01<1:00:51, 20.40s/it]

...

name: 할리베어 카페
time_dict : {'토': '추석 연휴10:00 - 20:00', '일': '09:00 - 20:00', '월': '임시공휴일10:00 - 20:00', '화': '개천절10:00 - 20:00', '수': '09:00 - 20:00', '목': '09:00 - 20:00', '금': '09:00 - 20:00'}
name: 제부콤마씨
Error : list index out of range


 99%|████████████████████████████████████████▊| 181/182 [49:51<00:16, 16.64s/it]

name: 제부도카페 캐리의집
time_dict : {'토': '추석 연휴10:00 - 21:00', '일': '10:00 - 21:00', '월': '임시공휴일10:00 - 21:00', '화': '개천절10:00 - 21:00', '수': '11:00 - 21:00', '목': '11:00 - 21:00', '금': '11:00 - 21:00'}


100%|█████████████████████████████████████████| 182/182 [50:07<00:00, 16.52s/it]

name: 카페타이타닉
time_dict : {'토': '10:00 - 22:00', '일': '10:00 - 22:00', '월': '10:00 - 20:00', '화': '10:00 - 20:00', '수': '정기휴무 (격주 수요일)', '목': '10:00 - 20:00', '금': '10:00 - 20:00'}


In [14]:
data_scrapped = {
    "name"     : name_list,
    "category" : category_list,
    "address"  : address_list,
    "time_mon" : time_mon_list, 
    "time_tue" : time_tue_list,
    "time_wed" : time_wed_list,
    "time_thr" : time_thr_list,
    "time_fri" : time_fri_list, 
    "time_sat" : time_sat_list,
    "time_sun" : time_sun_list,
    "tel"      : tel_list,
    "desc1"    : desc1_list,
    "desc2"    : desc2_list,
    "desc3"    : desc3_list,
}

In [15]:
df_scrapped = pd.DataFrame(data_scrapped)
df_scrapped.to_excel('./data/df_scrapped_gb_230930.xlsx')

In [16]:
df_scrapped

,name,category,address,time_mon,time_tue,time_wed,time_thr,time_fri,time_sat,time_sun,tel,desc1,desc2,desc3
0,을왕리해수욕장,"해수욕장,해변",인천 중구 용유서로302번길 16-15,,,,,,,,,찾아가는길서울→경인고속도로→월미도 가는 표지판→이를 따라 \n공장지대를 지나쳐 월미...,서울→경인고속도로→월미도 가는 표지판→이를 따라 \n공장지대를 지나쳐 월미도 선착장...,인천광역시 중구 을왕동에 있는 해수욕장. 인천광역시 중구 을왕동에 있는 해수욕장...
1,왕산해수욕장,"해수욕장,해변",인천 중구 을왕동,,,,,,,,,찾아가는길서울→영종대교(신공항고속도로)→용유.무의(이정표)쪽→해안도로→무의도→왕산해수욕장,서울→영종대교(신공항고속도로)→용유.무의(이정표)쪽→해안도로→무의도→왕산해수욕장,| 분류 | 레포츠 여행 | 성큼 다가온 여름이 반가운 건 해변에서 즐기는 캠핑 때...
2,영종도 마시안 해변,"해수욕장,해변",인천 중구 마시란로 118,,,,,,,,,,가까운곳에 이렇게 멋진 바다를 볼수있다니~\n주변에 카페들이 많아 더위도 시킬수있고...,
3,선녀바위해수욕장,"해수욕장,해변",인천 중구 을왕동 678-188,,,,,,,,,,바람쐬고 드라이브하기 좋아요\n휴가철인데 사람도 없고 한적한데 바다라 시원하기까지....,[개요] 선녀바위해수욕장은 해변에서 보이는 선녀바위와 기암들이 매력적인 곳으로 바다...
4,용유도해변,"해수욕장,해변",인천 중구 남북동,,,,,,,,,,좋아요 !,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,카페 이니씨오,"카페,디저트",경기 화성시 서신면 해안길 330 가동 1층,11:00 - 19:00,11:00 - 19:00,11:00 - 19:00,11:00 - 19:00,11:00 - 19:00,10:00 - 19:30,10:00 - 19:00,0507-1422-8062,결제수단지역화폐(카드),새로 오픈한 신상카페입니다.,영업 종료10:00에 영업 시작10시 0분에 영업 시작
178,할리베어 카페,"카페,디저트",경기 화성시 서신면 해안길 80 할리베어,임시공휴일10:00 - 20:00,개천절10:00 - 20:00,09:00 - 20:00,09:00 - 20:00,09:00 - 20:00,추석 연휴10:00 - 20:00,09:00 - 20:00,0507-1371-0247,찾아가는길제부도 바닷길 건너와서 왼쪽으로 200m 민트색 2층건물 입니다,제부도 오션뷰로 이국적 테마카페이며\n1층 30평 2층 70평 대규모 카페로1층 야...,영업 종료09:00에 영업 시작에 영업 시작
179,제부콤마씨,"카페,디저트",경기 화성시 서신면 해안길 344 1층,,,,,,,,,,,
180,제부도카페 캐리의집,"카페,디저트",경기 화성시 서신면 제부로 349,임시공휴일10:00 - 21:00,개천절10:00 - 21:00,11:00 - 21:00,11:00 - 21:00,11:00 - 21:00,추석 연휴10:00 - 21:00,10:00 - 21:00,031-355-3027,찾아가는길저희 제부도카페 캐리의 집은 제부도 바닷길이 열리는 \n제부도 입구에서 자...,안녕하세요 ^ _ ^ 제부도카페 캐리의 집 입니다. ~ ::)) \n저희 제부도 카...,영업 종료10:00에 영업 시작10시 0분에 영업 시작
